# [玉山人工智慧公開挑戰賽2019秋季賽,真相只有一個 -『信用卡盜刷偵測』](https://tbrain.trendmicro.com.tw/Competitions/Details/10)

## <font color=red>任務:預測某刷卡交易是否為盜刷</font>

### Task Schedule:
1. 讀取資料,將字串轉換成int
2. EDA(exploratory data analysis)
3. Feature engineering
4. 訓練模型,調整參數(預計使用lgb，速度較快)
5. 嘗試使用不同模型,做Ensamble(blending, stacking)
6. Anomaly detection

### 注意事項:
1. 因為test data和train data時間不相關,在驗證時採取前60天訓練61~90天驗證,但仍需小心時間差異造成的影響

### TODO:
1. **EDA(見下方詳細解釋）,找出不適合作為training feature的特徵,加以轉化成高級特徵或刪除**
2. **找data leakage**

3. Anomaly detection: 看這類的模型能不能取代lgb(似乎是不行，盜刷數據並沒有那麼Anomaly）,但可以嘗試將Anomaly結果當成新feature

### <font color=green>Results:</font>
* 不做處理,直接丟lgb訓練 leaderboard score:0.45


## 讀取,轉換字串成可以訓練的資料

In [25]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import math

%matplotlib inline
data_path = '../../data/raw_data'

random_seed = 2000

In [26]:
train_data_path = os.path.join(data_path,'train.zip')
train_data = pd.read_csv(train_data_path, encoding = "big5")

test_data_path = os.path.join(data_path,'test.zip')
test_data = pd.read_csv(test_data_path, encoding = "big5")

train_data_num = train_data.shape[0]
test_data_txkey = test_data['txkey'].copy()

train_data = train_data.sort_values(by=['bacno','locdt','loctm']).reset_index(drop=True)
label_data = train_data['fraud_ind'].copy()

all_data = pd.concat([train_data,test_data],axis=0).reset_index(drop=True)
print(all_data.index)
print(train_data.shape)
print(test_data.shape)
print(all_data.shape)

/home/kaiwen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


RangeIndex(start=0, stop=1943452, step=1)
(1521787, 23)
(421665, 22)
(1943452, 23)


In [27]:
train_data.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6413,1,117264,934.49,5,62,N,4,N,N,...,0,3,200000.0,275,53099,N,5817,102,0,1549254
1,6189,1,117264,939.19,5,62,Y,2,N,N,...,0,4,221428.0,317,90151,N,1463,102,0,1837177
2,6189,1,117264,1267.47,5,62,Y,2,N,N,...,0,25,212635.0,317,90151,N,1463,102,0,1859385
3,6231,1,117264,1017.37,5,62,N,5,N,N,...,0,30,200947.0,277,12726,N,5817,102,0,994333
4,6189,1,117264,613.81,5,62,N,4,N,N,...,0,34,150512.0,263,92571,N,5817,102,0,1639576


In [28]:
all_data.ecfg = all_data.ecfg.map({'N':0,'Y':1})
all_data.ovrlt = all_data.ovrlt.map({'N':0,'Y':1})
all_data.insfg = all_data.insfg.map({'N':0,'Y':1})
all_data.flbmk = all_data.flbmk.map({'N':0,'Y':1})
all_data.flg_3dsmk = all_data.flg_3dsmk.map({'N':0,'Y':1})
all_data.loctm = all_data.loctm.astype(int)
all_data = all_data.infer_objects()

# print(all_data.dtypes)
print('Missing value training data:\n',train_data.isna().sum()[train_data.isna().sum()>0])
print('Missing value testing data:\n',test_data.isna().sum()[test_data.isna().sum()>0])

## not neccessary to fill null value, since we use lgb model
all_data.flbmk = all_data.flbmk.fillna(value=all_data.flbmk.mean(skipna=True))
all_data.flg_3dsmk = all_data.flg_3dsmk.fillna(value=all_data.flg_3dsmk.mean(skipna=True))

Missing value training data:
 flbmk        12581
flg_3dsmk    12581
dtype: int64
Missing value testing data:
 flbmk        3715
flg_3dsmk    3715
dtype: int64


## Dirty Data

In [29]:
weird1 = (all_data['insfg']==1)&(all_data['iterm']==0)
print(weird1.value_counts())
print(all_data[weird1]['fraud_ind'].sum())

False    1943422
True          30
dtype: int64
0.0


In [30]:
binary_list=['ecfg','insfg','ovrlt','flbmk','flg_3dsmk']
category_list=['contp','etymd','hcefg','stocn','scity','stscd','csmcu']

## Feature engineering
* train & valid only（先不考慮test data)

In [ ]:
# for i in range(500):
#     print(i,all_data.groupby(['bacno']).get_group(i)[['ecfg','fraud_ind']])
# mean_df = all_data.groupby(['bacno'])['fraud_ind'].mean().reset_index()
# mean_df.columns = ['bacno', 'loctm_hr'+'_mean']
# all_data = pd.merge(all_data, mean_df, on='bacno', how='left')

# print(all_data[['bacno','locdt','loctm']])

# 該交易的歸戶帳號是否曾經被盜刷 0->沒 1->有 -1->無紀錄


# 該交易的歸戶帳號是否曾經被盜刷卻又復原
# 該交易的歸戶帳號是否第一次刷卡
# 該交易的歸戶帳號第幾次刷卡

# 該交易的卡號是否曾經被盜刷
# 該交易的卡號是否曾經被盜刷卻又復原
# 該交易的卡號是否第一次刷卡
# 該交易的卡號第幾次刷卡

# mean_df = all_data.groupby(['bacno']).apply(lambda s:s.mode()).reset_index()
# mean_df.columns = ['bacno', 'stocn'+'_mode']
# mean_df.fillna(-1,inplace=True)
# print(mean_df.stocn_mode.value_counts())
# all_data = pd.merge(all_data, mean_df, on='bacno', how='left')

# 消費國別是否跟自己所有消費的眾數不一樣
# 消費城市是否跟自己所有消費的眾數不一樣
# 消費地幣別是否跟自己所有消費的眾數不一樣
# 支付型態是否跟自己所有消費的眾數不一樣
# 分期期數是否跟自己所有消費的眾數不一樣

# 是否第一次網路消費且過去有非網路消費的經驗



In [ ]:
# data = pd.concat([df[:train_num], train_Y], axis=1)
# for c in df.columns:
#     mean_df = data.groupby([c])['SalePrice'].mean().reset_index()
#     mean_df.columns = [c, f'{c}_mean']
#     data = pd.merge(data, mean_df, on=c, how='left')
#     data = data.drop([c] , axis=1)


# all_data['howmany_cano'] = 
# all_data['howmany_txkey'] = 

## bacno刷卡頻率分佈

# all_data['fraud_before'] =
# all_data['fraud_last_time'] =

# 印出某個被盜刷的人的刷卡使用時間分佈


In [31]:
import pandas as pd
import os
import numpy as np
import math
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans

import time
import csv
random_seed = 20

def Kmeans_all_feature(x,n=1):
    stdsc = StandardScaler() 
    x = stdsc.fit_transform(x)

    clf = KMeans(n_clusters=n,random_state=random_seed)
    x2 = clf.fit_transform(x)
    x3 = clf.fit_predict(x)

    distance = np.zeros(x3.shape)
    for i in range(distance.shape[0]):
        distance[i] = x2[i,x3[i]]

    return distance

In [34]:
# kmeans_all_feature1 = Kmeans_all_feature(all_data.drop(columns='fraud_ind'),1)
# kmeans_all_feature3 = Kmeans_all_feature(all_data.drop(columns='fraud_ind'),3)
# kmeans_all_feature5 = Kmeans_all_feature(all_data.drop(columns='fraud_ind'),5)
all_data2 = all_data.drop(columns='fraud_ind')

center (3, 22)


In [42]:
print(x2.shape)
print(x3.shape)

(1943452, 3)
(1943452,)


In [43]:

# center = center[x2]
# print('center',center.shape)
# distance = ((x-center)**2).sum()

[[6.08686214 8.47145536 2.98552791]
 [6.2555605  9.02820318 4.36373747]
 [6.30010133 8.90267273 4.35000951]
 ...
 [5.92281808 8.53902671 2.89847152]
 [5.8982945  8.45682859 2.91590979]
 [5.42491824 8.52716664 3.06734698]]
[2.98552791 4.36373747 4.35000951 ... 2.89847152 2.91590979 3.06734698]
